In [4]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display
import nltk
from nltk import pos_tag
import timeit

In [5]:
import language_tool_python

In [6]:
tool = language_tool_python.LanguageTool('en-US')

In [7]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Vaishnavi D\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# !pip install gingerit

In [9]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i)
words_df = temp[:5000] 

In [10]:
tokens_tag = pos_tag(words_df)
dic_tag = {}
for tup in tokens_tag:
  if tup[1] not in dic_tag.keys():
    dic_tag[tup[1]] = [tup[0]]
  else:
    temp = dic_tag[tup[1]] + [tup[0]]
    dic_tag[tup[1]] = temp
# print(dic_tag.keys())
z = list(dic_tag.values())
words_df2 = []
for i in range(len(z)):
  words_df2.extend(z[i])
# print(words_df2)
words_df = words_df2

In [11]:
class poem_generation:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    def select_random_words(self,words_df):
        tic=timeit.default_timer()
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for select_random_words:",elapsed) 
        print("=========================================================")
        return words_array1

    def create_poems(self,arr):
        tic=timeit.default_timer()
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            poem[i-1]=' '.join(poem[i-1])
            # print(poem[i-1])
        print("=========================================================")
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for create_poems:",elapsed) 
        return poem

    def store_syllables(self,arr):
        tic=timeit.default_timer()
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for store_syllables:",elapsed)
        return a

    def Grammar_checker(self,text):
        tic=timeit.default_timer()
        error = 0
        poem_error = []
        tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for Grammar_checker:",elapsed)
        return poem_error

    def crossover(self,p1,p2,cross):
        tic=timeit.default_timer()
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
            child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for crossover:",elapsed)
        return child1,child2,child1_poem,child2_poem

    def tournament(self,Population,m):
        tic=timeit.default_timer()
        p = int(m/3)
        nn = m-p
        df = pd.DataFrame()
        selected_polulation = pd.DataFrame()
        Population.sort_values(by=['GrammarError'], inplace=True)
        ######################### when no syllables rule
        selected_polulation = Population.iloc[:nn]
        random_number = random.sample(list(Population.iloc[nn:].index),p)
        ######################### when syllables rule 
        # selected_polulation = Population[(Population["Syllables1"] == "5")&(Population["Syllables2"] == "6")&(Population["Syllables3"] == "6")&(Population["Syllables4"] == "5")]
        # Population2 = Population[~Population.isin(selected_polulation).all(1)]
        # z = len(selected_polulation.index)
        # n = nn - z
        # if z > nn:
        #  remove = z - nn
        #  for i in range(remove):
        #  selected_polulation.pop()
        # print("p:",p)
        # if p > len(list(Population2.index)):
                # print("Error")
        # random_number = random.sample(list(Population2.index),p)
    
        for i in random_number:
            df.append(Population.iloc[i])
        selected_polulation = pd.concat([selected_polulation,df],axis=0)
        selected_polulation = selected_polulation.reset_index(drop=True)
        print()
        print("SELECTED")
        display(selected_polulation)
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for tournament:",elapsed)
        return selected_polulation,selected_polulation["words_array"]   


    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            print()
            print("PARENTS")
            display(Pop_data)

            return Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        if len(words_array)%2 == 2:
            for i in range(0,len(words_array)-1):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        else:
            for i in range(0,len(words_array)-1,2):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
        child_data = pd.concat([child_data,words_array_child],axis=1)
        child_data = child_data.reset_index(drop=True)
        print()
        print("CHILD")
        display(child_data)
        return child_data

    def selection_from_child_parents(self,x,child_data,selected_Pop_data):
        reduce = (len(list(child_data.index)) + len(list(selected_Pop_data.index))) - x
        n = int(reduce/3)
        p = reduce - n
        if reduce > len(list(child_data.index)) + len(list(selected_Pop_data.index)):
            print("ERROR")
        else:
            if n > len(child_data):
                n = len(child_data)
                p = reduce - p
            if n == 0:
                n = 1
                p = 1
        d = pd.concat([selected_Pop_data,child_data],axis=0)
        d = d.reset_index(drop=True)
        # print("p:",p)
        # print("n:",n)
        random_number1 = random.sample(list(selected_Pop_data.index),p)
        random_number2 = random.sample(list(range(len(list(selected_Pop_data.index)),len(list(d.index)))),n)
        random_number1.extend(random_number2)
        print(random_number1)
        for i in random_number1:
          d.drop(i,axis=0,inplace = True)
        d = d.reset_index(drop=True)
        # print("NEW POPULATION")
        # display(d)
        return d
    
    def __init__(self,words_df,n,cross,epoch):
        tic_O=timeit.default_timer()
        Selection_number = int(round(0.85*n))
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        # assert n>=Selection_number, f"Cannot select {Selection_number} poems from total population of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        # assert 2*n >= Selection_number ,f"Cannot make initial population of {n} if only {Selection_number} parents are selected"

        Population_data = self.create_set_of_poems(n)
        for i in range(epoch):
            Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number)
            child_population = self.child_poems(selected_polulation_word_array,cross)
            final_population = self.selection_from_child_parents(n,child_population,Population_data)
            if i != epoch-1:
                print("Final Population")
                display(final_population)
                Population_data = final_population
            else:
                print("###################################################")
                print("Last Poems Standing")
                print("###################################################")
                display(final_population)
                toc_O=timeit.default_timer()
                elapsed_o = toc_O - tic_O #elapsed time in seconds
                print("Time for Total:",elapsed_o)




In [13]:
#4 (words_df,intial_population,crossover,epochs)3:
poem = poem_generation(words_df,20,2,10)

Time for select_random_words: 3.930000002583256e-05
Time for create_poems: 3.6399999999048305e-05
Time for store_syllables: 0.0004997000000912522


CalledProcessError: Command '['C:\\Program Files\\Common Files\\Oracle\\Java\\javapath\\java.exe', '-version']' returned non-zero exit status 1.

In [ ]:
print(list(range(5,8)))

[5, 6, 7]
